# Instagram Comments Cleaner
This notebook cleans the scraped Instagram comments data for semantic analysis

In [14]:
import pandas as pd
import re

# Load the scraped Instagram comments
dataframe = pd.read_csv('instagram_comments.csv')

In [15]:
# Initial analysis - see what we're working with
print("Original dataset info:")
print(f"Total comments: {len(dataframe)}")
print(f"Columns: {list(dataframe.columns)}")
print("\nFirst few rows:")
dataframe.head()

Original dataset info:
Total comments: 288
Columns: ['comment_id', 'username', 'text', 'created_at_utc', 'like_count']

First few rows:


,comment_id,username,text,created_at_utc,like_count
0,17942279211025487,nikkijblazaro,😢😢,2025-09-08 11:54:30+00:00,0
1,17916805491172606,mayyyaaa55,kasuhan na bakit kasi pinapatagal pa,2025-09-08 10:59:06+00:00,0
2,18073485872032680,_vhsg75,"Praying for you sir, na lagi kang healthy para...",2025-09-08 12:12:51+00:00,0
3,17858349186498231,elvzav,kaya nga dapat may makasuhan na ano pa bang ka...,2025-09-08 09:34:28+00:00,0
4,18036365528480441,ethelguarin,God bless you po Sir. Praying na gabayan ka at...,2025-09-08 16:39:16+00:00,0


In [16]:
# Show detailed statistics before cleaning
print("Before cleaning:")
dataframe.describe()

Before cleaning:


,comment_id,like_count
count,2.880000e+02,288.0
mean,1.806777e+16,0.0
std,1.528785e+14,0.0
min,1.784200e+16,0.0
25%,1.796902e+16,0.0
50%,1.806619e+16,0.0
75%,1.809613e+16,0.0
max,1.854162e+16,0.0


In [17]:
# Show sample of text content to understand what needs cleaning
print("Sample comment texts:")
for i, text in enumerate(dataframe['text'].head(10)):
    print(f"{i+1}. {text}")

Sample comment texts:
1. 😢😢
2. kasuhan na bakit kasi pinapatagal pa
3. Praying for you sir, na lagi kang healthy para magampanan  mo ng maayos ung trabaho na binigay sayo para mapabuti ang sambayanang Pilipino God bless!
4. kaya nga dapat may makasuhan na ano pa bang kailangan , ibalik ang pera
5. God bless you po Sir. Praying na gabayan ka at protektahan ka ni Lord para maiayos ang katiwalian at managot ang dapat managot sa DPWH. 🙏🙏🙏
6. Nakakahiya na talaga
7. Kasuhan na agad before they “escape” and hide their loot and bank accounts, issue hold departure order
8. Dapat yang mag asawa makulong at ibalik ang pera ng bayan
9. Yun ang problema wala nang tiawala ang tao sayo
10. Small but terrible 👏👏👏👏


In [18]:
# Start cleaning process
print("🧹 Starting Instagram comment cleaning...")

# 1. Remove comments with empty or null text
initial_count = len(dataframe)
dataframe = dataframe[dataframe['text'].notna()]
dataframe = dataframe[dataframe['text'].str.strip() != '']
print(f"Removed {initial_count - len(dataframe)} empty/null comments")

# 2. Remove comments that are just emojis (more comprehensive emoji patterns)
emoji_pattern = r'^[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF\U00002600-\U000027BF\U0001f900-\U0001f9ff\U0001f600-\U0001f64f\U0001f300-\U0001f5ff\U0001f680-\U0001f6ff\U0001f1e0-\U0001f1ff\U0001F190-\U0001F1FF\U0001F910-\U0001F96B\U0001F980-\U0001F997\U0001F9C0-\U0001F9C2\U0001F9D0-\U0001F9FF\U00002702-\U000027B0\u2600-\u26FF\u2700-\u27BF\s]*$'
before_emoji = len(dataframe)
dataframe = dataframe[~dataframe['text'].str.match(emoji_pattern, na=False)]
print(f"Removed {before_emoji - len(dataframe)} emoji-only comments")

# 3. Remove single-word comments (NEW)
def is_single_word(text):
    if pd.isna(text):
        return False
    # Split by whitespace and count meaningful words (not just punctuation)
    words = [word.strip() for word in str(text).split() if word.strip() and not word.strip() in '.,!?;:']
    return len(words) == 1

before_single_word = len(dataframe)
dataframe = dataframe[~dataframe['text'].apply(is_single_word)]
print(f"Removed {before_single_word - len(dataframe)} single-word comments")

# 4. Remove very short comments (less than 4 characters for better quality)
before_short = len(dataframe)
dataframe = dataframe[dataframe['text'].str.len() >= 4]
print(f"Removed {before_short - len(dataframe)} very short comments")

# 5. Remove comments that are just punctuation or symbols
punctuation_pattern = r'^[^\w\s]*$'
before_punct = len(dataframe)
dataframe = dataframe[~dataframe['text'].str.match(punctuation_pattern, na=False)]
print(f"Removed {before_punct - len(dataframe)} punctuation-only comments")

🧹 Starting Instagram comment cleaning...
Removed 3 empty/null comments
Removed 22 emoji-only comments
Removed 17 single-word comments
Removed 0 very short comments
Removed 0 punctuation-only comments


In [19]:
# 6. Remove common spam and low-quality patterns (ENHANCED)
spam_patterns = [
    r'^(dm me|dm|follow me|follow back|f4f|l4l|like4like|follow4follow)$',
    r'^(check my.*|visit my.*|link in bio|see my profile).*',
    r'^(buy.*|sale.*|discount.*|promo.*|offer.*)$',
    r'^(@\w+\s*)+$',  # Comments that are just mentions
    r'^(first|second|third|1st|2nd|3rd|last|early)$',  # Just ordinal numbers
    r'^\.\.\.$',  # Just dots
    r'^(yes|no|ok|okay|lol|haha|wow|omg|nice|good|bad|cool|awesome|amazing)$',  # Single reaction words
    r'^(love|like|want|need|get|see|look|watch|read|buy)$',  # Single action words
    r'^[0-9]+$',  # Just numbers
    r'^(x|xx|xxx|xxxx)$',  # Just x's
    r'^(true|false|real|fake|right|wrong)$',  # Single judgment words
]

before_spam = len(dataframe)
for pattern in spam_patterns:
    dataframe = dataframe[~dataframe['text'].str.match(pattern, case=False, na=False)]
print(f"Removed {before_spam - len(dataframe)} spam/low-quality comments")

# 7. Remove comments that are just repeated characters
repeated_char_pattern = r'^(.)\1+$'  # Same character repeated
before_repeated = len(dataframe)
dataframe = dataframe[~dataframe['text'].str.match(repeated_char_pattern, na=False)]
print(f"Removed {before_repeated - len(dataframe)} repeated character comments")

# 8. Remove comments from potential bot accounts (ENHANCED)
bot_patterns = r'(bot|spam|fake|auto|promo|sale|buy|shop|follow|like|comment|sub|subscribe).*\d{3,}$'
before_bots = len(dataframe)
dataframe = dataframe[~dataframe['username'].str.match(bot_patterns, case=False, na=False)]
print(f"Removed {before_bots - len(dataframe)} potential bot comments")

# 9. Remove comments that are just keyboard smashing (NEW)
def is_keyboard_smash(text):
    if pd.isna(text):
        return False
    text = str(text).lower()
    # Check for patterns like "asdfgh", "qwerty", "hjklm" etc.
    keyboard_patterns = [
        'qwerty', 'asdfgh', 'zxcvbn', 'hjklm', 'yuiop',
        'qwertyui', 'asdfghjk', 'zxcvbnm',
        'abcdefg', '1234567', 'aaaaaaa'
    ]
    return any(pattern in text for pattern in keyboard_patterns) and len(text) <= 15

before_keyboard = len(dataframe)
dataframe = dataframe[~dataframe['text'].apply(is_keyboard_smash)]
print(f"Removed {before_keyboard - len(dataframe)} keyboard smash comments")

Removed 0 spam/low-quality comments
Removed 0 repeated character comments
Removed 0 potential bot comments
Removed 0 keyboard smash comments


In [20]:
# 10. Remove duplicate comments (same text from same user)
before_duplicates = len(dataframe)
dataframe = dataframe.drop_duplicates(subset=['username', 'text'], keep='first')
print(f"Removed {before_duplicates - len(dataframe)} duplicate comments")

# 11. Remove comments that are just URLs or links
before_links = len(dataframe)
url_pattern = r'^https?://.*$|^www\..*$|^.*\.com.*$|^.*\.org.*$'
dataframe = dataframe[~dataframe['text'].str.match(url_pattern, na=False)]
print(f"Removed {before_links - len(dataframe)} URL-only comments")

# 12. Remove comments with excessive repeated characters (enhanced)
def has_excessive_repeats(text):
    if pd.isna(text):
        return False
    text = str(text)
    # Check if any character repeats more than 3 times consecutively (stricter)
    if re.search(r'(.)\1{3,}', text):
        return True
    # Check for excessive repeated words like "good good good good"
    words = text.lower().split()
    if len(words) > 1:
        for i in range(len(words) - 2):
            if words[i] == words[i+1] == words[i+2]:  # Same word 3+ times
                return True
    return False

before_repeats = len(dataframe)
dataframe = dataframe[~dataframe['text'].apply(has_excessive_repeats)]
print(f"Removed {before_repeats - len(dataframe)} comments with excessive repetition")

# 13. Remove comments that are just common social media expressions (NEW)
social_expressions = [
    r'^(tag|tagged|tagging).*$',
    r'^(share|shared|sharing).*$',
    r'^(story|stories).*$',
    r'^(post|posted|posting).*$',
    r'^(caption|captions).*$',
    r'^(photo|pic|picture|image).*$',
    r'^(video|vid|vlog).*$',
]

before_social = len(dataframe)
for pattern in social_expressions:
    dataframe = dataframe[~dataframe['text'].str.match(pattern, case=False, na=False)]
print(f"Removed {before_social - len(dataframe)} generic social media expressions")

# 14. Remove comments that are too generic/meaningless (NEW)
generic_comments = [
    r'^(this|that|it|what|when|where|how|why|who)$',
    r'^(same|different|similar|other|another)$',
    r'^(me|you|him|her|them|us|we|they)$',
    r'^(my|your|his|her|their|our)$',
    r'^(here|there|now|then|today|yesterday|tomorrow)$',
]

before_generic = len(dataframe)
for pattern in generic_comments:
    dataframe = dataframe[~dataframe['text'].str.match(pattern, case=False, na=False)]
print(f"Removed {before_generic - len(dataframe)} overly generic comments")

Removed 0 duplicate comments
Removed 0 URL-only comments
Removed 20 comments with excessive repetition
Removed 0 generic social media expressions
Removed 0 overly generic comments


In [21]:
# Clean up text content (enhanced normalization)
def clean_text(text):
    if pd.isna(text):
        return text
    
    text = str(text)
    
    # Remove excessive punctuation (more than 2 consecutive)
    text = re.sub(r'[.]{3,}', '...', text)  # Multiple dots to ellipsis
    text = re.sub(r'[!]{2,}', '!', text)    # Multiple exclamations to one
    text = re.sub(r'[?]{2,}', '?', text)    # Multiple questions to one
    
    # Remove excessive whitespace
    text = ' '.join(text.split())
    
    # Remove leading/trailing whitespace and punctuation
    text = text.strip(' .,!?;:')
    
    # Normalize common internet slang to more readable forms
    text = re.sub(r'\bu\b', 'you', text, flags=re.IGNORECASE)
    text = re.sub(r'\bur\b', 'your', text, flags=re.IGNORECASE)
    text = re.sub(r'\br\b', 'are', text, flags=re.IGNORECASE)
    
    return text

dataframe['text'] = dataframe['text'].apply(clean_text)
print("✅ Normalized text content (cleaned punctuation and slang)")

✅ Normalized text content (cleaned punctuation and slang)


In [22]:
# 15. Advanced content quality filtering (NEW)
def is_meaningful_comment(text):
    """Check if comment has meaningful content for semantic analysis"""
    if pd.isna(text):
        return False
    
    text = str(text).strip().lower()
    
    # Must have at least 2 words
    words = [w for w in text.split() if w.isalpha()]
    if len(words) < 2:
        return False
    
    # Check for actual sentence structure (has both nouns/verbs and connects words)
    connecting_words = ['and', 'or', 'but', 'so', 'because', 'when', 'where', 'how', 'why', 'what', 'that', 'this', 'is', 'are', 'was', 'were', 'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'should', 'could', 'can', 'may', 'might']
    has_structure = any(word in text for word in connecting_words)
    
    # Check if it's not just a list of adjectives
    adjectives_only = ['good', 'bad', 'nice', 'cool', 'awesome', 'amazing', 'great', 'terrible', 'horrible', 'beautiful', 'ugly', 'hot', 'cold', 'big', 'small']
    is_just_adjectives = all(word in adjectives_only for word in words)
    
    return has_structure and not is_just_adjectives

before_meaningful = len(dataframe)
dataframe = dataframe[dataframe['text'].apply(is_meaningful_comment)]
print(f"Removed {before_meaningful - len(dataframe)} comments lacking meaningful content")

# 16. Remove comments that are just questions without context (NEW)
context_less_questions = [
    r'^\?+$',  # Just question marks
    r'^(what|why|how|when|where|who)\?*$',  # Single question words
    r'^(really|seriously|actually)\?*$',  # Single incredulous words
    r'^(huh|eh|hmm|uhh|umm)\?*$',  # Thinking sounds
]

before_questions = len(dataframe)
for pattern in context_less_questions:
    dataframe = dataframe[~dataframe['text'].str.match(pattern, case=False, na=False)]
print(f"Removed {before_questions - len(dataframe)} context-less questions")

Removed 92 comments lacking meaningful content
Removed 0 context-less questions


In [23]:
# Final cleanup - remove any rows that became empty after text cleaning
before_final = len(dataframe)
dataframe = dataframe[dataframe['text'].str.len() > 0]
print(f"Final cleanup: removed {before_final - len(dataframe)} empty entries")

# Reset index
dataframe.reset_index(drop=True, inplace=True)

Final cleanup: removed 0 empty entries


In [24]:
# Show cleaning results
print("\n📊 CLEANING SUMMARY:")
print(f"✅ Final dataset: {len(dataframe)} comments")
print(f"📉 Removed: {initial_count - len(dataframe)} comments ({((initial_count - len(dataframe)) / initial_count * 100):.1f}%)")
print("\nAfter cleaning:")
dataframe.describe()


📊 CLEANING SUMMARY:
✅ Final dataset: 134 comments
📉 Removed: 154 comments (53.5%)

After cleaning:


,comment_id,like_count
count,1.340000e+02,134.0
mean,1.808316e+16,0.0
std,1.537480e+14,0.0
min,1.784200e+16,0.0
25%,1.798849e+16,0.0
50%,1.807404e+16,0.0
75%,1.810838e+16,0.0
max,1.854162e+16,0.0


In [25]:
# Show sample of cleaned comments
print("\n🔍 Sample of cleaned comments:")
for i, row in dataframe.head(10).iterrows():
    print(f"{i+1}. @{row['username']}: {row['text'][:100]}{'...' if len(row['text']) > 100 else ''}")


🔍 Sample of cleaned comments:
1. @_vhsg75: Praying for you sir, na lagi kang healthy para magampanan mo ng maayos ung trabaho na binigay sayo p...
2. @elvzav: kaya nga dapat may makasuhan na ano pa bang kailangan , ibalik ang pera
3. @ethelguarin: God bless you po Sir. Praying na gabayan ka at protektahan ka ni Lord para maiayos ang katiwalian at...
4. @iamgatch69: Kasuhan na agad before they “escape” and hide their loot and bank accounts, issue hold departure ord...
5. @cewster_: Tapos bigla akong na gising.. Panaginip lang pala 😂
6. @lorabatesebastian: Words are easy. Kelan ang aksyon
7. @kura6eru: The untouchable will be touched🤣
8. @jerwin6330: nice para naman ibang issue nanaman ibato ng mga DDS sa pangulo hanap ulit sila ng paninira sa gobye...
9. @mackay_75: I hope so , because we are watching the hearing and it seems like … not right
10. @johnglenn274: TUMBUKIN NA AGAD ANG PINAKA PUNO. Hindi naman KIKILOS yang mga career officials na yan kung ealang b...


In [26]:
# Check for any potential issues in cleaned data
print("\n🔍 Quality check:")
print(f"Comments with very long text (>500 chars): {len(dataframe[dataframe['text'].str.len() > 500])}")
print(f"Unique usernames: {dataframe['username'].nunique()}")
print(f"Average comment length: {dataframe['text'].str.len().mean():.1f} characters")
print(f"Comments with mentions (@): {len(dataframe[dataframe['text'].str.contains('@', na=False)])}")
print(f"Comments with hashtags (#): {len(dataframe[dataframe['text'].str.contains('#', na=False)])}")


🔍 Quality check:
Comments with very long text (>500 chars): 2
Unique usernames: 133
Average comment length: 110.7 characters
Comments with mentions (@): 1
Comments with hashtags (#): 1


In [27]:
# Save the cleaned dataset
dataframe.to_csv('cleaned_instagram_comments.csv', index=False)
print("💾 Saved cleaned comments to 'cleaned_instagram_comments.csv')")

💾 Saved cleaned comments to 'cleaned_instagram_comments.csv')
